In [1]:
import numpy as np
import torch
from torch import nn, div, square, norm
from torch.nn import functional as F
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time

In [2]:
datapath = 'ml-1m/'
seed = 42
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)

In [3]:
RATINGS_PATH = 'ml-1m/ratings.dat'
USERS_PATH = 'ml-1m/users.dat'
ITEMS_PATH = 'ml-1m/movies.dat'
def load_movielens_1m():
    ratings_cols = ['user_id', 'item_id', 'rating', 'timestamp']
    ratings = pd.read_csv(RATINGS_PATH, sep='::', names=ratings_cols, engine='python', encoding='latin-1')

    users_cols = ['user_id', 'gender', 'age', 'occupation', 'zip_code']
    users = pd.read_csv(USERS_PATH, sep='::', names=users_cols, engine='python', encoding='latin-1')

    items_cols = ['item_id', 'title', 'genres']
    items = pd.read_csv(ITEMS_PATH, sep='::', names=items_cols, engine='python', encoding='latin-1')

    items['genres'] = items['genres'].str.split('|')
    
    return ratings, users, items

ratings_df, users_df, items_df = load_movielens_1m()
num_users, num_items = users_df['user_id'].max(), items_df['item_id'].max()
# reindex to 0-based
users_df['user_id'] -= 1
items_df['item_id'] -= 1
ratings_df['user_id'] -= 1
ratings_df['item_id'] -= 1


In [7]:
ratings_df

,user_id,item_id,rating,timestamp
0,0,1192,5,978300760
1,0,660,3,978302109
2,0,913,3,978301968
3,0,3407,4,978300275
4,0,2354,5,978824291
...,...,...,...,...
1000204,6039,1090,1,956716541
1000205,6039,1093,5,956704887
1000206,6039,561,5,956704746
1000207,6039,1095,4,956715648


In [18]:
train_ratings, test_ratings = train_test_split(ratings_df,
                                           test_size=0.1,
                                           random_state=seed)
train_ratings
utility_matrix = train_ratings.pivot(index='user_id', columns='item_id', values='rating')

# Optionally fill missing ratings with 0 or NaN
utility_matrix = utility_matrix.fillna(0)

test_utility_matrix = train_ratings.pivot(index='user_id', columns='item_id', values='rating')

# Optionally fill missing ratings with 0 or NaN
test_utility_matrix = utility_matrix.fillna(0)

In [19]:
test_utility_matrix

item_id,0,1,2,3,4,5,6,7,8,9,...,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951
user_id,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# class MF(object):
#     def __init__(self, Y, K, lam = 0.1, Xinit = None, Winit = None, learning_rate = 0.5, max_iter = 1000, print_every = 100):
#         self.Y = Y # represents the utility matrix
#         self.K = K #
#         self.lam = lam # regularization parameter
#         self.learning_rate = learning_rate # for gradient descent
#         self.max_iter = max_iter # maximum number of iterations
#         self.print_every = print_every # print loss after each a few iters
#         self.n_users = int(np.max(Y[:, 0])) + 1
#         self.n_items = int(np.max(Y[:, 1])) + 1
#         self.n_ratings = Y.shape[0] # number of known ratings
#         self.X = np.random.randn(self.n_items, K) if Xinit is None else Xinit
#         self.W = np.random.randn(K, self.n_users) if Winit is None else Winit
#         self.b = np.random.randn(self.n_items) # item biases
#         self.d = np.random.randn(self.n_users) # user biases

#     def loss(self):
#         L = 0
#         for i in range(self.n_ratings):
#             n, m, rating = int(self.Y[i, 0]), int(self.Y[i, 1]), self.Y[i, 2]
#             L += 0.5*(self.X[m].dot(self.W[:, n]) + self.b[m] + self.d[n] - rating)**2
#         L /= self.n_ratings

#         return L + 0.5*self.lam*(np.sum(self.X**2) + np.sum(self.W**2))
    
#     def updateXb(self):
#         for m in range(self.n_items):
#             # get all users who rated item m and get the corresponding ratings
#             ids = np.where(self.Y[:, 1] == m)[0] # row indices of items m
#             user_ids, ratings = self.Y[ids, 0].astype(np.int32), self.Y[ids, 2]
#             Wm, dm = self.W[:, user_ids], self.d[user_ids]
#             for i in range(30): # 30 iteration for each sub problem
#                 xm = self.X[m]
#                 error = xm.dot(Wm) + self.b[m] + dm - ratings
#                 grad_xm = error.dot(Wm.T)/self.n_ratings + self.lam*xm
#                 grad_bm = np.sum(error)/self.n_ratings
#                 # gradient descent
#                 self.X[m] -= self.learning_rate*grad_xm.reshape(-1)
#                 self.b[m] -= self.learning_rate*grad_
                
#     def updateWd(self): # and d
#         for n in range(self.n_users):
#             # get all items rated by user n, and the corresponding ratings
#             ids = np.where(self.Y[:,0] == n)[0] # row indices of items rated by user n
#             item_ids, ratings = self.Y[ids, 1].astype(np.int32), self.Y[ids, 2]
#             Xn, bn = self.X[item_ids], self.b[item_ids]
#             for i in range(30): # 30 iteration for each sub problem
#                 wn = self.W[:, n]
#                 error = Xn.dot(wn) + bn + self.d[n] - ratings
#                 grad_wn = Xn.T.dot(error)/self.n_ratings + self.lam*wn
#                 grad_dn = np.sum(error)/self.n_ratings
#                 # gradient descent
#                 self.W[:, n] -= self.learning_rate*grad_wn.reshape(-1)
#                 self.d[n] -= self.learning_rate*grad_dn
    
#     def fit(self):
#         for it in range(self.max_iter):
#             self.updateWd()
#             self.updateXb()
#             if (it + 1) % self.print_every == 0:
#                 rmse_train = self.evaluate_RMSE(self.Y)
#                 print("iter = %d, loss = %.4f, RMSE train = %.4f"%(it + 1, self.loss(), rmse_train))
                
#     def pred(self, u, i):
#         """
#         predict the rating of user u for item i
#         """
#         u, i = int(u), int(i)
#         pred = self.X[i, :].dot(self.W[:, u]) + self.b[i] + self.d[u]# + bias
#         return max(0, min(5, pred)) # pred should be between 0 and 5 in MoviesLen
    
#     def evaluate_RMSE(self, rate_test):
#         n_tests = rate_test.shape[0] # number of test
#         SE = 0 # squared error
#         for n in range(n_tests):
#             pred = self.pred(rate_test[n, 0], rate_test[n, 1])
#             SE += (pred - rate_test[n, 2])**2
#         RMSE = np.sqrt(SE/n_tests)
#         return RMSE

In [22]:
R_df = utility_matrix
R_test = test_utility_matrix
# Convert to numpy array
R = R_df.to_numpy()
num_users, num_items = R.shape
R_test = R_test.to_numpy()
# Mask of where ratings exist (nonzero entries)
observed_test = R_test > 0
observed = R > 0
# Hyperparameters
K = 20  # number of latent features
steps = 30  # number of iterations
alpha = 0.005  # learning rate
lambda_ = 0.02  # regularization strength

# Initialization
P = np.random.normal(0, 0.1, (num_users, K))
Q = np.random.normal(0, 0.1, (num_items, K))
b_u = np.zeros(num_users)
b_i = np.zeros(num_items)
mu = np.sum(R) / np.count_nonzero(R)

In [23]:
# Training using gradient descent
for step in range(steps):
    for u in range(num_users):
        for i in range(num_items):
            if observed[u, i]:
                # Prediction and error
                pred = mu + b_u[u] + b_i[i] + np.dot(P[u], Q[i])
                e_ui = R[u, i] - pred

                # Update biases
                b_u[u] += alpha * (e_ui - lambda_ * b_u[u])
                b_i[i] += alpha * (e_ui - lambda_ * b_i[i])

                # Update latent factors
                P[u] += alpha * (e_ui * Q[i] - lambda_ * P[u])
                Q[i] += alpha * (e_ui * P[u] - lambda_ * Q[i])

    if step % 1 == 0:
        error = 0
        for u in range(num_users):
            for i in range(num_items):
                if observed_test[u, i]:
                    pred = mu + b_u[u] + b_i[i] + np.dot(P[u], Q[i])
                    error += (R_test[u, i] - pred) ** 2

        rmse = np.sqrt(error / np.count_nonzero(R_test))
        print(f"Step {step}, RMSE: {rmse:.4f}")

Step 0, RMSE: 0.9313
Step 1, RMSE: 0.9120
Step 2, RMSE: 0.9043
Step 3, RMSE: 0.8996
Step 4, RMSE: 0.8959
Step 5, RMSE: 0.8922
Step 6, RMSE: 0.8877
Step 7, RMSE: 0.8818
Step 8, RMSE: 0.8743
Step 9, RMSE: 0.8660
Step 10, RMSE: 0.8574
Step 11, RMSE: 0.8489
Step 12, RMSE: 0.8406
Step 13, RMSE: 0.8327
Step 14, RMSE: 0.8250
Step 15, RMSE: 0.8177
Step 16, RMSE: 0.8107
Step 17, RMSE: 0.8040
Step 18, RMSE: 0.7978
Step 19, RMSE: 0.7920
Step 20, RMSE: 0.7865
Step 21, RMSE: 0.7814
Step 22, RMSE: 0.7767
Step 23, RMSE: 0.7724
Step 24, RMSE: 0.7683
Step 25, RMSE: 0.7645
Step 26, RMSE: 0.7610
Step 27, RMSE: 0.7577
Step 28, RMSE: 0.7546
Step 29, RMSE: 0.7518
